In [ ]:
!pip install -e .

In [ ]:
!pip install --upgrade torch

In [1]:
import torch
import torch.onnx

In [2]:
torch.__version__


'1.11.0+cu102'

In [3]:
from uberduck_ml_dev.models.tacotron2 import Tacotron2, Tacotron2ForwardIsInfer, DEFAULTS
from uberduck_ml_dev.data_loader import prepare_input_sequence
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.models.common import griffin_lim, MelSTFT

In [4]:
vals = DEFAULTS.values()
vals.update(cudnn_enabled=False)
hp = HParams(**vals)
model = Tacotron2ForwardIsInfer(hp)

Not using any style tokens


In [29]:
# sd = torch.load("model_store/tacotron2-trebek-arpa-740-01-02-21.pt", map_location="cpu")

In [30]:
# model.load_state_dict(sd)

In [5]:
input_sequence = prepare_input_sequence(["The quick brown fox jumped over the lazy dog and then pooped in my mouth."], cpu_run=True, arpabet=True)
inputs, input_lengths = input_sequence

In [6]:
from typing import List

input_: List[torch.Tensor] = [inputs, input_lengths, torch.tensor([0]), torch.tensor([])]

In [7]:
# output = model.inference(input_)
# model(input_)

In [8]:
torch.onnx.export(
    model,
    (input_,),
    "test.onnx", 
    verbose=True, 
    opset_version=15,
    dynamic_axes=dict(
        mel_out={0: "batch_size", 2: "output_length"}, 
        mel_postnet={0: "batch_size", 2: "output_length"}, 
        gate={0: "batch_size", 1: "output_length"}, 
        attn={0: "batch_size", 1: "output_length", 2: "sequence_length"},
        input={0: "batch_size", 1: "sequence_length"},                 
    ),
    input_names=["input", "input_lengths", "speaker_id", "gst"],
    output_names=["mel_out", "mel_postnet", "gate", "attn"], #, "output_frames"],
)

RuntimeError: Tried to trace <__torch__.uberduck_ml_dev.models.tacotron2.DecoderForwardIsInfer object at 0x55813430ce90> but it is not part of the active trace. Modules that are called during a trace must be registered as submodules of the thing being traced.

In [13]:
!rm test.onnx

In [12]:
import onnx

In [15]:
onnx_model = onnx.load("test.onnx")

In [16]:
onnx.checker.check_model(onnx_model)

In [18]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.7 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [19]:
import onnxruntime as ort

In [20]:
ort_session = ort.InferenceSession("test.onnx")

In [27]:
outputs = ort_session.run(None, dict(
    input=input_[0].numpy(),
    input_lengths=input_[1].numpy(),
    # speaker_id=input_[2].numpy(),
    # gst=torch.tensor([0]).numpy(),
))

In [31]:
outputs

[array([[[ 0.01026152, -0.00902704,  0.01409964, ...,  0.02946579,
          -0.03711459, -0.02627717],
         [ 0.01374556, -0.01479521,  0.01587596, ...,  0.03938702,
          -0.04116368, -0.03862724],
         [ 0.02579391,  0.00696362,  0.01729412, ...,  0.03686604,
          -0.04322954, -0.03794964],
         ...,
         [ 0.02635956, -0.03205585,  0.01594216, ...,  0.0248093 ,
          -0.03902123, -0.05191857],
         [ 0.02494591, -0.02939685,  0.02393676, ...,  0.02861363,
          -0.02562611, -0.04096306],
         [ 0.03352709, -0.02950083,  0.01288793, ...,  0.01620274,
          -0.01906958, -0.03075045]]], dtype=float32)]

In [19]:
torch.equal(torch.sum(torch.tensor([2,3,3, -9, 1])), torch.tensor(0))

True

In [18]:
torch.sum(torch.tensor([2, 3, 3, -9]))

tensor(-1)

In [21]:
torch.rand(3, 3).shape

torch.Size([3, 3])

In [29]:
if torch.eq(torch.tensor(torch.rand(3, 3).shape[1]), torch.tensor(4)):
    print("true")

In [25]:
?torch.eq